In [ ]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=922907d3fffa49bb3cc587c989f1f9ac856fa18e1e

In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [ ]:
#load pdf files
loader = PyPDFDirectoryLoader("/content/sample_data/Data/")
data = loader.load()

In [ ]:
print(data)

[Document(page_content='See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/255657987\nDeﬁning mental health and mental illness\nArticle  · Januar y 2009\nCITATIONS\n9READS\n141,176\n2 author s, including:\nSome o f the author s of this public ation ar e also w orking on these r elat ed pr ojects:\nCultural P sychiatr y View pr oject\nDiversifying the medic al curriculum  View pr oject\nNisha Dogr a\nUniv ersity of L eicester\n152 PUBLICA TIONS \xa0\xa0\xa02,847  CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nAll c ontent f ollo wing this p age was uplo aded b y Nisha Dogr a on 20 May 2014.\nThe user has r equest ed enhanc ement of the do wnlo aded file.', metadata={'source': '/content/sample_data/Data/mental_health_Document.pdf', 'page': 0}), Document(page_content='Deﬁning mental health and\nmental illness\nSharon Leighton and Nisha Dogra\nKey features \x7fDiscussion of the terminological confusion that exists in relation to issu

In [ ]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)


In [ ]:
len(text_chunks)

52

In [ ]:
#get the third chunk
text_chunks[2]

Document(page_content='terms used are readily understood by others. The scale of the problem and access to services isoutlined. We then discuss stigma generally, explore the reasons for it and possible sequelae, andthen consider how this relates to children. Finally, interventions to reduce stigma are brie ﬂy\npresented. As mentioned in the Introduction, where possible we have referred speci ﬁcally to the\nliterature relating to children but where this is limited we have drawn from the wider literatureto highlight key issues.\nThe chapter begins with an exercise which provides a practical context for the theoretical\ncontent and should be borne in mind as you read, and answered once you have ﬁnished the\nchapter.\nBox 1.1\nExercise\nGeneral questions\n\x7f What words or images do you associate with the following terms:\n\x7f Mental health\n\x7f Mental health problems\x7f Mental illness\x7f Mental disorder\n\x7f What sorts of problems do people experience that could be described as ment

In [ ]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-large")


In [ ]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [ ]:
#connect to google drive
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="/content/drive/MyDrive/Model&Data/zephyr-7b-beta.Q4_K_M.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [ ]:
query = "how do I improve my mental health"

In [ ]:
qa.run(query)

" to improve your mental health, you need to focus on promoting subjective wellbeing, developing cognitive, affective, and relational abilities, achieving individual and collective goals consistent with justice, while considering cultural and developmental contexts. This involves interacting with others and the environment in a positive way. It's also essential to recognize that mental health is interdependent with physical and social functioning."

In [ ]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Llama.generate: prefix-match hit


Answer:  While it's true that there are interventions to reduce stigma surrounding mental illness, preventing mental illness from developing in the first place is also important. This involves addressing underlying risk factors such as poverty, trauma, and social exclusion, as well as promoting protective factors such as strong relationships and a sense of purpose and meaning in life. Parents, schools, and communities can all play a role in fostering resilience and supporting children's mental health. Additionally, early identification and intervention for mental health concerns can help prevent more serious problems from developing. By taking a holistic approach to promoting mental health and preventing mental illness, we can make a difference in the lives of children and their families.
